In [ ]:
import sapien.core as sapien
from Tools import misc, ForwardKinematics, SimWorker, NumForwardDynamicsDer
import numpy as onp
import jax.numpy as np
from jax import jit, jacfwd, jacrev, grad
from ilqr import ILQR
from datetime import datetime
from tqdm.notebook import trange
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
DEBUG = False

sim = sapien.Engine()
renderer = sapien.OptifuserRenderer()
sim.set_renderer(renderer)
render_controller = sapien.OptifuserController(renderer)

stabled = False


def create_scene(timestep, visual):
    s = sim.create_scene([0, 0, 0])
    s.add_ground(-1)
    s.set_timestep(timestep)

    loader = s.create_urdf_loader()
    loader.fix_root_link = True
    if visual:
        loader.collision_is_visual = True
        s.set_ambient_light([0.5, 0.5, 0.5])
        s.set_shadow_light([0, 1, -1], [0.5, 0.5, 0.5])

    # build
    robot = loader.load("../../../assets/Arm/panda.urdf")

    return s, robot


sim_timestep = 1 / 60
optim_timestep = 1 / 60
s0, robot = create_scene(sim_timestep, True)

render_controller.set_camera_position(-5, 0, 0)
render_controller.set_current_scene(s0)

if not stabled:
    for _ in range(3000):
        s0.step()
    stabled = True

In [ ]:
num_deri = NumForwardDynamicsDer(robot, sim_timestep)
fk = ForwardKinematics(robot)
sim_worker = SimWorker(robot, create_scene, optim_timestep, DEBUG)

In [ ]:
# render_controller.show_window()
# while True:
#     u = onp.random.randn(robot.dof) * 0.5
#     u = onp.clip(u, -1, 1)
#     robot.set_qf(u)
#     s0.step()
#     s0.update_render()
#     render_controller.render()

u_range = np.array([[-10] * robot.dof, [10] * robot.dof])
pred_time = 1.3
horizon = int(pred_time / optim_timestep) + 1
per_iter = 3


def smooth_abs(x, alpha):
    return np.mean((alpha ** 2) * (np.cosh(x / alpha) - 1))


robo_pose = robot.get_root_pose()


@jit
def final_cost(x, alpha=0.3):
    # add base pose
    x = np.concatenate((x, robo_pose.p, robo_pose.q))

    cart_pos = fk.fk(x).reshape(-1, 3)[:, :3]
    end_effector_pos = cart_pos[-3]

    target_height = x[-1] + 1.2

#     term1 = smooth_abs(target_height - end_effector_pos[2], alpha)
    term1 = np.abd(target_height - end_effector_pos[2])

    return term1 * 10

limits = robot.get_qlimits().reshape(2, -1)
limits_sum = np.abs((limits[1] - limits[0]))/2
limits_mid = np.mean(limits, axis=0)

@jit
def running_cost(x, u, alpha=0.5):
    term1 = np.sum(smooth_abs(u / 5, alpha)) / horizon * 2
    
    dist2limit = np.abs(x - limits_mid) / limits_sum
    # term2 = np.sum(smooth_abs(dist2limit / limits_sum, alpha))
    term2 = smooth_abs(dist2limit, 1) / horizon
    
    return term1 + term2

In [ ]:

ilqr = ILQR(final_cost, running_cost, None, u_range, horizon, per_iter, num_deri, sim_worker, DEBUG)

In [ ]:
state = misc.get_state(robot)
num_x = len(state)
num_u = robot.dof
dof = robot.dof

# prep seq
x_seq = []
u_seq = []
pack_seq = []

bak_pack = robot.pack()

for i in range(horizon):
    u = onp.random.randn(robot.dof) * 0.5
    u = onp.clip(u, -1, 1)

    x = misc.get_state(robot)
    pack = robot.pack()

    x_seq.append(x)
    pack_seq.append(pack)
    u_seq.append(u)

    robot.set_qf(u)
    s0.step()
robot.unpack(bak_pack)

In [ ]:
# records
ctrl_record = []
x_record = []
u_record = []
run_cost_record = []
f_cost_record = []

# plots
fig, axs = plt.subplots(3,figsize=(15,9))
plt.ion()

fig.show()
fig.canvas.draw()

total = []
f_ax, r_ax, t_ax = axs
PLOT_LEN = 100

render_controller.show_window()
for i in trange(1000):
    x_seq, u_seq, pack_seq = ilqr.predict(x_seq, u_seq, pack_seq)

    u = u_seq[0]

    robot.set_qf(u)
    s0.step()
    s0.update_render()
    render_controller.render()

    new_x = misc.get_state(robot)
    new_pack = robot.pack()

    x_seq[0] = new_x
    pack_seq[0] = new_pack
    
    #record
    f_cost = final_cost(x_seq[-1])
    run_cost = onp.sum([running_cost(x, u) for x, u in zip(x_seq[:-1], u_seq[:-1])])
    cost = f_cost + run_cost
    
    ctrl_record.append(u)
    x_record.append(x_seq)
    u_record.append(u_seq)
    run_cost_record.append(run_cost)
    f_cost_record.append(f_cost)

    # plot
    f_ax.clear()
    r_ax.clear()
    t_ax.clear()

    total.append(cost)

    y_lim = np.max(total[-PLOT_LEN:])
    f_ax.set_ylim(0, y_lim)
    r_ax.set_ylim(0, y_lim)
    f_ax.set_ylim(0, y_lim)

    x_lim = max(0, i - PLOT_LEN)
    f_ax.set_xlim(x_lim, i)
    r_ax.set_xlim(x_lim, i)
    t_ax.set_xlim(x_lim, i)


    f_ax.plot(f_cost_record)
    r_ax.plot(run_cost_record)
    t_ax.plot(total)


    fig.canvas.draw()
    fig.show()


In [ ]:
record = {
    'ctrl_record': ctrl_record,
    'x_record': x_record,
    'u_record': u_record,
    'run_cost_record': run_cost_record,
    'f_cost_record': f_cost_record
}

np.save("record" + str(datetime.utcnow()), record)